## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [38]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path

In [39]:
def submit_to_ars(m,ars_url='https://ars.ci.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid,ars_url='https://ars.ci.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results'
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown'
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [42]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            file_read = path.join(root, name)
            dir_name = (os.path.splitext(os.path.basename(root))[0])
            print(file_read)
            
            filename = (os.path.splitext(os.path.basename(file_read))[0])
            print(filename)
            with open(file_read,'r') as inf:
                query = json.load(inf)
                
                kcresult = submit_to_ars(query)
                
                result_status = retrieve_ars_results(kcresult)
                
        
                dict_workflows[filename] = kcresult
                            
                sleep(200)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=687eeec8-021b-4772-975e-b0d0d3ab5a23
Running
Running
kp-cam Running 0
Running
ara-unsecret Running 0
Done
kp-cohd ARS Error 0
Running
kp-icees Running 0
Running
ara-aragorn Running 0
Running
kp-textmining Running 0
Running
kp-openpredict Running 0
Done
ara-explanatory Done 0
Done
ara-improving Done 5
Running
kp-chp Running 0
Running
kp-molecular Running 0
Done
kp-genetics Done 0
Running
ara-arax Running 0
Running
ara-bte Running 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.8_EGFR_simple.json
A.8_EGFR_simple
https://arax.ncats.io/?source=ARS&id=835bc4b7-6eb8-4edc-bd81-24d1ca7bd606
Running
Running
kp-cam Running 0
Running
ara-bte Running 0
Running
ara-aragorn Running 0
Done
ara-improving ARS Error 0
Done
kp-genetics Done 0
Running
kp-openpredict Running 0
Running
kp-textmining Running 0
Done
kp-molecular Done 0
Done
ara-explana

https://arax.ncats.io/?source=ARS&id=9cc6176b-06be-4e5d-b5b4-0c986f84131f
Running
Running
ara-unsecret Running 0
Done
kp-icees ARS Error 0
Running
ara-bte Running 0
Running
ara-aragorn Running 0
Running
kp-textmining Running 0
Running
ara-improving Running 0
Error
kp-molecular Error 0
Done
ara-explanatory Done 0
Running
kp-cam Running 0
Error
kp-chp Error 0
Running
ara-arax Running 0
Error
kp-cohd Error 0
Running
kp-openpredict Running 0
Error
kp-genetics Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowD/D.4_tryptophan-kynurenine-with-curie.json
D.4_tryptophan-kynurenine-with-curie
https://arax.ncats.io/?source=ARS&id=5704a821-fef6-4fc0-ac83-b91c75172f2d
Running
Running
ara-unsecret Running 0
Error
kp-textmining Error 0
Running
ara-arax Running 0
Running
kp-openpredict Running 0
Error
kp-cohd Error 0
Done
ara-explanatory Done 0
Running
ara-aragorn Running 0
Done
kp-icees ARS Error 0
Running
kp-cam Running 0
Error
kp-molecular Error 0
Error
ara-bte Error 0
Ru

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [43]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1 687eeec8-021b-4772-975e-b0d0d3ab5a23
Done
Done
kp-cam Done 0
Error
ara-unsecret Error 0
Done
kp-cohd ARS Error 0
Done
kp-icees Done 0
Done
ara-aragorn Done 314
Done
kp-textmining Done 0
Done
kp-openpredict Done 0
Done
ara-explanatory Done 0
Done
ara-improving Done 5
Done
kp-chp Done 10
Done
kp-molecular Done 99
Done
kp-genetics Done 0
Done
ara-arax Done 68
Done
ara-bte Done 40
A.8_EGFR_simple 835bc4b7-6eb8-4edc-bd81-24d1ca7bd606
Done
Done
kp-cam Done 0
Done
ara-bte Done 0
Error
ara-aragorn Error 0
Done
ara-improving ARS Error 0
Done
kp-genetics Done 0
Done
kp-openpredict Done 0
Done
kp-textmining Done 0
Done
kp-molecular Done 0
Done
ara-explanatory Done 0
Error
ara-unsecret Error 0
Error
kp-cohd Error 0
Done
ara-arax Done 336
Error
kp-chp Error 0
Done
kp-icees Done 0
A.2_RHOBTB2_twohop 8d2e0126-c53d-45d4-a914-8e9195b2e068
Done
Done
ara-bte Done 0
Done
ara-aragorn Done 0
Done
kp-cam Done 0
Error
kp-genetics Error 0
Done
ara-arax Done 441
Error
kp-molecular Error 0
Done
kp-tex

kp-molecular Error 0
Done
kp-icees ARS Error 0
Error
ara-aragorn Error 0
Error
ara-arax Error 0
Error
kp-genetics Error 0
Error
ara-unsecret Error 0
Done
kp-openpredict Done 0
Error
kp-cam Error 0
Error
kp-textmining Error 0
C.2_ChemSubstances_related_to_GeneSet 2b53977a-a9c9-4076-ae3a-da2f182566ba
Done
Error
ara-improving Error 0
Error
kp-genetics Error 0
Error
ara-bte Error 0
Error
kp-textmining Error 0
Error
kp-cohd Error 0
Error
kp-cam Error 0
Error
ara-arax Error 0
Done
ara-explanatory Done 0
Done
kp-openpredict Done 0
Done
kp-chp Done 0
Error
ara-unsecret Error 0
Done
kp-icees ARS Error 0
Error
kp-molecular Error 0
Error
ara-aragorn Error 0
C.1_ChemSubstances_related_to_Disease 9a762505-079a-4231-a3dd-47ff5f94a4df
Done
Error
ara-unsecret Error 0
Error
kp-textmining Error 0
Done
ara-improving ARS Error 0
Done
kp-icees Done 0
Error
kp-cohd Error 0
Error
kp-chp Error 0
Error
kp-molecular Error 0
Done
kp-genetics Done 0
Done
kp-openpredict Done 0
Done
ara-explanatory Done 0
Error
ara

In [45]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k].items():
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi
B.3_DILI-one-hop-from-genes
B.4_one-hop-gene-biological-process-or-activity_trapi
C.1_ChemSubstances_related_to_Disease
C.2_ChemSubstances_related_to_GeneSet
D.1_parkinsons-crohns
D.2_ssri-heart-disease
D.3_ssri-heart-disease-one-hop
D.4_tryptophan-kynurenine-with-curie
D.5_tryptophan-kynurenine
D.6_metformin-ferritin


In [46]:
df

,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=42e6398a-...,https://arax.ncats.io/?source=ARS&id=8d2e0126-...,https://arax.ncats.io/?source=ARS&id=687eeec8-...,https://arax.ncats.io/?source=ARS&id=835bc4b7-...,https://arax.ncats.io/?source=ARS&id=bf19fc60-...,https://arax.ncats.io/?source=ARS&id=76c7660e-...,https://arax.ncats.io/?source=ARS&id=51038248-...,https://arax.ncats.io/?source=ARS&id=661149dc-...,https://arax.ncats.io/?source=ARS&id=9cc6176b-...,https://arax.ncats.io/?source=ARS&id=201a44fc-...,...,https://arax.ncats.io/?source=ARS&id=458d508e-...,https://arax.ncats.io/?source=ARS&id=edfa630f-...,https://arax.ncats.io/?source=ARS&id=9a762505-...,https://arax.ncats.io/?source=ARS&id=2b53977a-...,https://arax.ncats.io/?source=ARS&id=f5ba2b9a-...,https://arax.ncats.io/?source=ARS&id=7b1ab8b1-...,https://arax.ncats.io/?source=ARS&id=a08618ed-...,https://arax.ncats.io/?source=ARS&id=5704a821-...,https://arax.ncats.io/?source=ARS&id=7ef13171-...,https://arax.ncats.io/?source=ARS&id=8b950ba5-...
kp-openpredict,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
kp-icees,No Results,ARS Error,No Results,No Results,No Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,...,No Results,No Results,No Results,ARS Error,ARS Error,ARS Error,No Results,ARS Error,ARS Error,ARS Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error
ara-bte,Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-cohd,Error,Error,ARS Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
ara-improving,Results,Results,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,...,ARS Error,No Results,ARS Error,Error,Results,Results,Results,Results,Results,Results
kp-genetics,No Results,Error,No Results,No Results,No Results,Error,Error,Error,Error,Error,...,No Results,No Results,No Results,Error,Error,Error,No Results,Error,Error,Error
ara-arax,Results,Results,Results,Results,Error,No Results,No Results,No Results,No Results,No Results,...,Results,Results,No Results,Error,Results,Error,Results,Error,Error,Error
ara-aragorn,No Results,No Results,Results,Error,Error,No Results,No Results,No Results,No Results,No Results,...,Results,Results,No Results,Error,Error,Error,No Results,Results,Error,No Results


<br>

## Converting the Pk's to hyperlink

<br>

In [47]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [48]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [49]:
df.sort_index(inplace=True)

In [50]:
df

,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results,No Results,Results,Error,Error,No Results,No Results,No Results,No Results,No Results,...,Results,Results,No Results,Error,Error,Error,No Results,Results,Error,No Results
ara-arax,Results,Results,Results,Results,Error,No Results,No Results,No Results,No Results,No Results,...,Results,Results,No Results,Error,Results,Error,Results,Error,Error,Error
ara-bte,Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,Results,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,...,ARS Error,No Results,ARS Error,Error,Results,Results,Results,Results,Results,Results
ara-unsecret,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-chp,No Results,No Results,Results,Error,Error,Error,Error,Error,Error,Error,...,Error,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results
kp-cohd,Error,Error,ARS Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-genetics,No Results,Error,No Results,No Results,No Results,Error,Error,Error,Error,Error,...,No Results,No Results,No Results,Error,Error,Error,No Results,Error,Error,Error


<br>

### Highlight the cells In Excel

<br>

In [51]:
def highlight(v):
    if v=='Results':
        return 'background-color: %s' % 'green'
    elif v=='Error':
        return 'background-color: %s' % 'red'
    elif v == 'No Results':
        return 'background-color: %s' % 'yellow'
    elif v == 'ARS Error':
        return 'background-color: %s' % 'blue'
    elif v == 'Unknown':
        return 'background-color: %s' % 'magneta'
    else:
        return

In [52]:
styled = df.style.applymap(highlight)

In [53]:
styled

,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results,No Results,Results,Error,Error,No Results,No Results,No Results,No Results,No Results,No Results,Error,Results,Results,No Results,Error,Error,Error,No Results,Results,Error,No Results
ara-arax,Results,Results,Results,Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,Error,Results,Results,No Results,Error,Results,Error,Results,Error,Error,Error
ara-bte,Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,Results,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,Error,ARS Error,No Results,ARS Error,Error,Results,Results,Results,Results,Results,Results
ara-unsecret,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-chp,No Results,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results
kp-cohd,Error,Error,ARS Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-genetics,No Results,Error,No Results,No Results,No Results,Error,Error,Error,Error,Error,Error,Error,No Results,No Results,No Results,Error,Error,Error,No Results,Error,Error,Error


In [74]:
styled.to_excel('ara_worklow_progress_tracker_Aug12.xlsx')

In [ ]:
### Save as excel sheet
#df.to_excel('ara_worklow_progress_tracker_Aug5.xlsx', sheet_name = 'Progress_Tracker')

In [ ]:
### save all the ids for ara submission
#with open('ara_worklow_IDs_Aug9.txt', 'w') as f:
#    print(dict_workflows, file=f)

<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

In [73]:
styled

,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results,No Results,Results,Error,Error,No Results,No Results,No Results,No Results,No Results,No Results,Error,Results,Results,No Results,Error,Error,Error,No Results,Results,Error,No Results
ara-arax,Results,Results,Results,Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,Error,Results,Results,No Results,Error,Results,Error,Results,Error,Error,Error
ara-bte,Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,Results,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,Error,ARS Error,No Results,ARS Error,Error,Results,Results,Results,Results,Results,Results
ara-unsecret,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-chp,No Results,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results
kp-cohd,Error,Error,ARS Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-genetics,No Results,Error,No Results,No Results,No Results,Error,Error,Error,Error,Error,Error,Error,No Results,No Results,No Results,Error,Error,Error,No Results,Error,Error,Error


In [56]:
df

,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results,No Results,Results,Error,Error,No Results,No Results,No Results,No Results,No Results,...,Results,Results,No Results,Error,Error,Error,No Results,Results,Error,No Results
ara-arax,Results,Results,Results,Results,Error,No Results,No Results,No Results,No Results,No Results,...,Results,Results,No Results,Error,Results,Error,Results,Error,Error,Error
ara-bte,Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,Results,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,...,ARS Error,No Results,ARS Error,Error,Results,Results,Results,Results,Results,Results
ara-unsecret,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-chp,No Results,No Results,Results,Error,Error,Error,Error,Error,Error,Error,...,Error,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results
kp-cohd,Error,Error,ARS Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-genetics,No Results,Error,No Results,No Results,No Results,Error,Error,Error,Error,Error,...,No Results,No Results,No Results,Error,Error,Error,No Results,Error,Error,Error


<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

In [57]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [58]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [59]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
wks_name = 'Workflow Progress Tracker August 12'
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Workflow Progress Tracker August 12' id:2006211480>

In [ ]:
spreadsheet_key

In [60]:
import gspread
import pandas as pd
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *

In [61]:
pwd

'/Users/priyash/Documents/GitHub/minihackathons/Notebooks'

In [62]:
gc = gspread.service_account(filename='/Users/priyash/Documents/GitHub/minihackathons/Notebooks/araxworkflowprogresstesting-2632632db8be.json')
sh = gc.open("workflow_progress_tracker").sheet1

In [63]:
sh

<Worksheet 'Workflow Progress Tracker August 12' id:2006211480>

In [64]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:W16', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(1,0,0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

In [65]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:W15', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.5, 0.0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 2006211480,
       'startRowIndex': 1,
       'endRowIndex': 16,
       'startColumnIndex': 1,
       'endColumnIndex': 23}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 1}}}}}}},
  {},
  {}]}

In [66]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:W15', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['No Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0.75, 0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 2006211480,
       'startRowIndex': 1,
       'endRowIndex': 15,
       'startColumnIndex': 1,
       'endColumnIndex': 23}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.49803922}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 2006211480,
       'startRowIndex': 1,
       'endRowIndex': 16,
       'startColumnIndex': 1,
       'endColumnIndex': 23}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 1}}}}}}},
  {},
  {

In [67]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:W15', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['ARS Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.75, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 2006211480,
       'startRowIndex': 1,
       'endRowIndex': 15,
       'startColumnIndex': 1,
       'endColumnIndex': 23}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 0.7490196,
         'green': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 2006211480,
       'startRowIndex': 1,
       'endRowIndex': 15,
       'startColumnIndex': 1,
       'endColumnIndex': 23}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': True},
       'back

In [68]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:W15', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['Unknown']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 2006211480,
       'startRowIndex': 1,
       'endRowIndex': 15,
       'startColumnIndex': 1,
       'endColumnIndex': 23}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'ARS Error'}]},
      'format': {'backgroundColor': {'green': 0.7490196, 'blue': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.7490196,
         'blue': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 2006211480,
       'startRowIndex': 1,
       'endRowIndex': 15,
       'startColumnIndex': 1,
       'endColumnIndex': 23}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       'textFormat': {'bold'

In [69]:
# Select a range
cell_list = sh.range('B16:W16')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'Workflow Progress Tracker August 12'!B16:W16",
 'updatedRows': 1,
 'updatedColumns': 22,
 'updatedCells': 22}

In [72]:
df

,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results,No Results,Results,Error,Error,No Results,No Results,No Results,No Results,No Results,...,Results,Results,No Results,Error,Error,Error,No Results,Results,Error,No Results
ara-arax,Results,Results,Results,Results,Error,No Results,No Results,No Results,No Results,No Results,...,Results,Results,No Results,Error,Results,Error,Results,Error,Error,Error
ara-bte,Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,Results,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,...,ARS Error,No Results,ARS Error,Error,Results,Results,Results,Results,Results,Results
ara-unsecret,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-chp,No Results,No Results,Results,Error,Error,Error,Error,Error,Error,Error,...,Error,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results
kp-cohd,Error,Error,ARS Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-genetics,No Results,Error,No Results,No Results,No Results,Error,Error,Error,Error,Error,...,No Results,No Results,No Results,Error,Error,Error,No Results,Error,Error,Error
